# scikit-learnのトレーニング♨

## 環境準備、共通的データ処理 編
- [環境準備](#環境準備)
  - [インストール](#インストール)
  - [インポート](#インポート)
- [共通処理](#共通処理)
  - [モデルの生成](#モデルの生成)
  - [勾配降下法アルゴリズムの選択](#勾配降下法アルゴリズムの選択)
  - [過学習の抑止](#過学習の抑止)
  - [モデルの保存と復元](#モデルの保存と復元)

## [目次](TableOfContents.ipynb)

## 参考
開発基盤部会 Wiki
- データマイニング（DM）- Python - DL  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20Python%20-%20DL

## 環境準備

### インストール

#### 基本的なライブラリ

In [ ]:
!pip install numpy
!pip install pandas
!pip install seaborn
!pip install statsmodels

#### scikit-learn

In [ ]:
!pip install scikit-learn

#### tensorflow

In [ ]:
!pip install -U tensorflow

#### keras

In [ ]:
!pip install -U keras

#### opencv

In [ ]:
!pip install opencv-python

### インポート

#### 基本的なライブラリ

In [ ]:
import io
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### scikit-learn

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix as cm
from sklearn.model_selection import train_test_split

#### tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
print(tf.__version__)

#### keras

In [ ]:
import keras
print(keras.__version__)

# モデル定義
from keras.models import Model, Sequential, model_from_json
from keras.layers import Dense, Input, Activation, Flatten, Dropout, LSTM
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras.optimizers import SGD, Adam
# その他
from keras.applications.vgg16 import VGG16
from keras.utils import np_utils

#### その他

In [ ]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## 共通処理

### 勾配降下法アルゴリズムの選択
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E6%B7%B1%E5%B1%A4%E5%AD%A6%E7%BF%92%E3%81%AE%E3%83%86%E3%82%AF%E3%83%8B%E3%83%83%E3%82%AF#ma318ba7

#### SGD
（Stochastic Gradient Descent）、確率的勾配降下法
- 勾配が最も急な向きに勾配を下る手法
- 学習率の調整が難しいため自動調整のアルゴリズムが必要。
- lr：学習率（0以上のfloat

In [ ]:
optimizer = optimizers.SGD(lr=0.01)

#### MomentumSGD
- SGDに慣性（加速・減速）の概念を加えた最適化手法｡
- SGDのコードに「momentum」（慣性項の係数（0以上のF）を追加。
- 谷間での無駄な往復を減らし、早く収束するが、  
加速しているため、極小値付近で止まり難い。

In [ ]:
optimizer = optimizers.SGD(lr=0.01, momentum=0.9)

#### NesterovAG
（Nesterov Accelerated Gradient）
- 極小値付近でブレーキをかける機能を加えたアルゴリズム
- [MomentumSGD](#MomentumSGD)のコードに「nesterov=True」を追加。

In [ ]:
optimizer = optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)

#### AdaGrad
- 動いた量が増えたら更新が緩やかにする。
- [NesterovAG](#NesterovAG)のブレーキ機能を学習率の減衰によって実現
- デフォルト・パラメタが推奨
  - lr ：学習率（0以上のfloat
  - epsilon：分母の最小値（0以上のfloat
  - decay ：学習率減衰（0以上のfloat

In [ ]:
#optimizer = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
optimizer = optimizers.Adagrad()

#### RMSprop
（Root Mean Square Propagation）
- [AdaGrad](#AdaGrad)の改良版
  - 学習率の減衰が穏やかになる機能を追加
  - 引数に「rho」（指数移動平均）を追加。
- 学習率以外はデフォルト・パラメタが推奨

In [ ]:
# optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
optimizer = optimizers.RMSprop(lr=0.001)

#### AdaDelta
- [Adagrad](#Adagrad)の改良版の[RMSprop](#RMSprop)に､もうひと工夫加えたようなアルゴリズム
- デフォルト・パラメタが推奨

In [ ]:
# optimizer = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
optimizer = optimizers.Adadelta()

#### Adam
（ADAptive Moment estimation）
- [MomentumSGD](#MomentumSGD)系の慣性項と[AdaGrad](#AdaGrad)系の学習率の  
両方の自動調整の機能を持ち合わせたアルゴリズム
- beta_1 ：MomentumSGDのmomentum
- beta_2 ：AdaGradのrho
- amsgrad：AMSGradを適用するかどうか。

In [ ]:
optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

### モデルの生成

In [ ]:
# ...

### 過学習の抑止

#### 早期終了（early stopping）

##### EarlyStoppingクラス
- monitor  
監視する値
- patience  
指定エポック数の間、改善がない場合、学習を停止
- mode  
収束判定（auto: 自動, min: 最小時, max: 最大時）
- verbose  
標準出力で通知をするか否か

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=0)
#hist = model.fit(..., callbacks=[es]) # EarlyStoppingを適用

##### ModelCheckpointクラス
検証誤差が最も低い状態のモデルを保存
- monitor  
監視する値（デフォルト: val_loss）
- mode  
収束判定（auto: 自動, min: 最小時, max: 最大時）
- verbose  
標準出力で通知をするか否か
- filepath  
モデルを保存するファイルパス
- save_best_only  
最良の状態のモデルのみを保存するかどうか（bool）

In [ ]:
mc = ModelCheckpoint(monitor='val_loss', mode='min', verbose=1, filepath='./dl4', save_best_only=True)
#hist = model.fit(..., callbacks=[mc]) # ModelCheckpointを適用

#### ドロップアウト（Dropout）
最終層の一つ前に以下を追加。

In [ ]:
# 無効化比率0.5のDropout
#model.add(Dropout(rate=0.5))

#### バッチ正規化（Batch Normalization）
畳み込み層とプーリング層の間に以下を追加。

In [ ]:
#model.add(BatchNormalization())

#### モデルの評価

###### 学習履歴を表示

In [47]:
def plot_history_loss(hist):
    plt.plot(hist.history['loss'],label="loss for training")
    plt.plot(hist.history['val_loss'],label="loss for validation")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='best')
    plt.show()
    
#hist = model.fit(x_train, ...
#plot_history_loss(hist)

### モデルの保存と復元

#### モデルの保存

In [ ]:
# モデルはjson形式
json_string = model.to_json()
with open('mnist.model', 'w') as f:
    f.write(json_string)
# パラメータはhdf5形式
model.save_weights('param.hdf5')

#### モデルの復元
復元後、再度コンパイルが必要になる。

In [ ]:
# モデルはjson形式
with open('mnist.model', 'r') as f:
    json_string = f.read()
model = model_from_json(json_string)
# パラメータはhdf5形式
model.load_weights('param.hdf5')